In [1]:
# 套件匯入
import pandas as pd
import numpy as np
import os
import re

import talib
from talib import abstract

ModuleNotFoundError: ignored

In [ ]:
# 列出所有 TW Stock 檔案名稱
filenames = [f for f in os.listdir('../Try crawl/data/') if re.search('^Stock_TW', f)]
filenames.pop(-1)
print(len(filenames))
filenames

30


['Stock_TW_1301.TW.csv',
 'Stock_TW_1303.TW.csv',
 'Stock_TW_1304.TW.csv',
 'Stock_TW_1305.TW.csv',
 'Stock_TW_1308.TW.csv',
 'Stock_TW_1309.TW.csv',
 'Stock_TW_1312.TW.csv',
 'Stock_TW_1313.TW.csv',
 'Stock_TW_1314.TW.csv',
 'Stock_TW_1326.TW.csv',
 'Stock_TW_2002.TW.csv',
 'Stock_TW_2006.TW.csv',
 'Stock_TW_2010.TW.csv',
 'Stock_TW_2014.TW.csv',
 'Stock_TW_2023.TW.csv',
 'Stock_TW_2027.TW.csv',
 'Stock_TW_2028.TW.csv',
 'Stock_TW_2030.TW.csv',
 'Stock_TW_2031.TW.csv',
 'Stock_TW_2038.TW.csv',
 'Stock_TW_2603.TW.csv',
 'Stock_TW_2605.TW.csv',
 'Stock_TW_2606.TW.csv',
 'Stock_TW_2609.TW.csv',
 'Stock_TW_2615.TW.csv',
 'Stock_TW_2617.TW.csv',
 'Stock_TW_2636.TW.csv',
 'Stock_TW_2637.TW.csv',
 'Stock_TW_5608.TW.csv',
 'Stock_TW_5609.TWO.csv']

In [ ]:
# 再確認一下檔案長度
for filename in filenames:
    df = pd.read_csv(f'../Try crawl/data/{filename}')
    lendf = len(df)
    if lendf != 2816:
        print(f'The length of {filename} is {lendf}')

The length of Stock_TW_2637.TW.csv is 2589


In [ ]:
# 建立檔案存檔位置
if os.path.exists('afdata') is False:
    os.makedirs('afdata')

In [ ]:
# 乖離率
def BIAS(close, timeperiod=20):
    if isinstance(close,np.ndarray):
        pass
    else:
        close = np.array(close)
    MA = talib.MA(close,timeperiod=timeperiod)
    return (close-MA)/MA

In [ ]:
# 移動平均平行線差指標
# def AMA(stockStat):
#     return talib.MA(stockStat['dma'],  timeperiod=10)

# https://github.com/jealous/stockstats/blob/master/stockstats.py
# df['dma'] = df['close_10_sma'] - df['close_50_sma']

def AMA(df):
    return talib.MA(talib.SMA(df['close'], timeperiod=10)-talib.SMA(df['close'], timeperiod=50),  timeperiod=10)

In [ ]:
# 心理線指標
def PSY(priceData, period):
    difference = priceData[1:].values - priceData[:-1].values
    difference = np.append(0, difference)
    difference_dir = np.where(difference > 0, 1, 0)
    psy = np.zeros((len(priceData),))
    psy[:period] *= np.nan
    for i in range(period, len(priceData)):
        psy[i] = (difference_dir[i-period+1:i+1].sum()) / period
    return psy*100

# https://www.codeleading.com/article/72563130863/ 有英文註解

In [ ]:
# 區間震盪線
def DPO(close):
    p = talib.MA(close, timeperiod=11)
    p.shift()
    return close-p

In [ ]:
# 十字過濾線指標
def VHF(close):
    LCP = talib.MIN(close, timeperiod=28)
    HCP = talib.MAX(close, timeperiod=28)
    NUM = HCP - LCP
    pre = close.copy()
    pre = pre.shift()
    DEN = abs(close-close.shift())
    DEN = talib.MA(DEN, timeperiod=28)*28
    return NUM.div(DEN)

In [ ]:
# 相對活力指數
def RVI(df):
    close = df.close
    open = df.open
    high = df.high
    low = df.low
    X = close-open+2*(close.shift()-open.shift())+2*(close.shift(periods=2)-open.shift(periods=2))*(close.shift(periods=3)-open.shift(periods=3))/6
    Y = high-low+2*(high.shift()-low.shift())+2*(high.shift(periods=2)-low.shift(periods=2))*(high.shift(periods=3)-low.shift(periods=3))/6
    Z = talib.MA(X, timeperiod=10)*10
    D = talib.MA(Y, timeperiod=10)*10
    return Z/D

In [ ]:
for filename in filenames:
    
    # 讀檔案
    df = pd.read_csv(f'../Try crawl/data/{filename}')
    
    # 日期改至 2011-01-01 開始
    df = df[df['Date'].between('2011-01-01', '2021-06-30')]
    
    # 丟掉多餘的 columns, Date 換成 index
    df = df.drop(['Dividends', 'Stock Splits'], axis='columns').set_index('Date')
    
    # 將 columns 改成小寫以便之後 talib function 使用
    df.columns = df.columns.map(lambda x:x.lower())
    
    # All columns type change to float
    df = df.astype('float')
    
    # 全部技術指標
    ta_list = talib.get_functions()

    for x in ta_list:

        try:

            # x 為技術指標的代碼，透過迴圈填入，再透過 eval 計算出 output
            output = eval('abstract.'+x+'(df)')

            # 如果輸出是一維資料，幫這個指標取名為 x 本身；多維資料則不需命名
            output.name = x.lower() if type(output) == pd.core.series.Series else None

            # 透過 merge 把輸出結果併入 df DataFrame
            df = pd.merge(df, pd.DataFrame(output), left_on = df.index, right_on = output.index)
            df = df.set_index('key_0')

        except:

            print(x)
            
    # 日期特徵值
    datetime_series = pd.DatetimeIndex(df.index)
    
    
    df = pd.concat([df, pd.DataFrame(datetime_series.year.rename('year')).set_index(df.index),
                    pd.DataFrame(datetime_series.month.rename('month')).set_index(df.index),
                    pd.DataFrame(datetime_series.day.rename('day')).set_index(df.index),
                    pd.DataFrame(datetime_series.isocalendar().week.rename('weekofyear')).set_index(df.index),
                    pd.DataFrame(datetime_series.dayofyear.rename('dayofyear')).set_index(df.index),
                    pd.DataFrame(datetime_series.isocalendar().day.rename('dayofweek')).set_index(df.index)], axis=1)
    
    # 加入六個股票常用指標
    df = pd.concat([df, pd.DataFrame(BIAS(df['close'], 20), columns=['bias']).set_index(df.index),
                    pd.DataFrame(AMA(df), columns=['ama']),
                    pd.DataFrame(PSY(df['close'], 24), columns=['psy']).set_index(df.index),
                    pd.DataFrame(DPO(df['close']), columns=['dpo']),
                    pd.DataFrame(VHF(df['close']), columns=['vhf']),
                    pd.DataFrame(RVI(df), columns=['rvi'])], axis=1)
    
    print(f'The shape of AF_{filename}:')
    print(df.shape)
    
     # 存檔
    df.to_csv(f'afdata/AF_{filename}')
    print(f'Save file :　AF_{filename}')
    print('-'*100)

MAVP


c:\users\march\anaconda3\envs\env_pca\lib\site-packages\pandas\core\reshape\merge.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


The shape of AF_Stock_TW_1301.TW.csv:
(2566, 191)
Save file :　AF_Stock_TW_1301.TW.csv
----------------------------------------------------------------------------------------------------
MAVP
The shape of AF_Stock_TW_1303.TW.csv:
(2566, 191)
Save file :　AF_Stock_TW_1303.TW.csv
----------------------------------------------------------------------------------------------------
MAVP
The shape of AF_Stock_TW_1304.TW.csv:
(2566, 191)
Save file :　AF_Stock_TW_1304.TW.csv
----------------------------------------------------------------------------------------------------
MAVP
The shape of AF_Stock_TW_1305.TW.csv:
(2566, 191)
Save file :　AF_Stock_TW_1305.TW.csv
----------------------------------------------------------------------------------------------------
MAVP
The shape of AF_Stock_TW_1308.TW.csv:
(2566, 191)
Save file :　AF_Stock_TW_1308.TW.csv
----------------------------------------------------------------------------------------------------
MAVP
The shape of AF_Stock_TW_1309.TW.csv:
(2